# Fitting Encoding Models with NEMS Library

In this tutotial we show how to easily train models built using the [Neural Encoding Model System (NEMS)](https://github.com/LBHB/NEMS) library. Here, we show how to train cross-validated linear-nonlinear STRF models.

In [2]:
import numpy as np

import naplib as nl
from naplib.model_selection import KFold

import nems

In [3]:
from nems import Model
from nems.layers import STRF, DoubleExponential, StateGain
from nems.models import LN_STRF

In [4]:
filepath = '../out_structs/out_struct_sample.mat'
data = nl.io.import_outstruct(filepath)
print(f'This data has {len(data)} trials')

This data has 18 trials


In [5]:
# look at the shape of the spectrograms we have
print(data['aud'][0].shape) # (time * frequency)

(6047, 23)


In [6]:
# look at the shape of the responses we have
print(data['resp'][0].shape) # (time * channels)

# for simplicity and time-saving, let's only use the first 3 channels
data['resp'] = [x[:,:3] for x in data['resp']]
print(data['resp'][0].shape) # (time * channels)


(6047, 30)
(6047, 3)


In [7]:
# normalize responses
data['resp'] = nl.preprocessing.normalize(data=data)

## Define and fit a simple Linear-Nonlinear (LN) model with NEMS

This example model consists of a STRF component followed by a double-exponential nonlinearity.

In [8]:
# fitting options (quick fitting for the sake of this tutorial notebook)
options = {'options': {'maxiter': 2, 'ftol': 1e-1}}

### Fit the model using cross-validation

In [10]:
kfold = nl.model_selection.KFold(5) # 5-fold cross validation
predictions = []

for i, (data_train, data_test) in enumerate(kfold.split(data)):
    print(i)
    spec_train = np.concatenate(data_train['aud'], axis=0) # shape (time * frequency)
    spec_test = np.concatenate(data_test['aud'], axis=0) # shape (time * frequency)
    resp_train = np.concatenate(data_train['resp'], axis=0) # shape (time * channels)
    resp_test = np.concatenate(data_test['resp'], axis=0) # shape (time * channels)
    
    # define LN model
    model = Model()
    model.add_layers(
        STRF(shape=(25,23)),    # Full-rank STRF, 25 temporal bins x 23 spectral channels
        DoubleExponential(shape=(3,)) # Double-exponential nonlinearity
    )
    
    model.fit(input=spec_train, target=resp_train, fitter_options=options)
    
    predictions.append(model.predict(input=spec_test))

    

0
Epoch 0


../NEMS/nems/layers/nonlinearity.py:202: RuntimeWarning: overflow encountered in exp
  output = base + amplitude * np.exp(-np.exp(inner_exp))


        Iteration 0, error is: 1.01941962...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
1
Epoch 0
        Iteration 0, error is: 1.01720607...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
2
Epoch 0
        Iteration 0, error is: 1.02602287...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
3
Epoch 0
        Iteration 0, error is: 1.01905902...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
4
Epoch 0
        Iteration 0, error is: 1.02291236...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH


In [11]:
predictions_full = np.concatenate(predictions, axis=0)
resp_full = np.concatenate(data['resp'], axis=0)
predictions_full.shape, resp_full.shape

((100006, 3), (100006, 3))